In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-23dc494d-c1b6-854c-9bc7-b6aaf3ae3d4e)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import clear_output
%cd '/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC'
%run Unet_2D.ipynb
%run preprocessing_2D.ipynb
%run otherUnets.ipynb
%run convNext_vs2.ipynb
clear_output()

In [ ]:
!pip install pytorch-lightning
!pip install timm
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob
import torch
import timm.optim
import torch.nn as nn
from timm.data.loader import create_loader
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from fastprogress import master_bar, progress_bar
from sklearn.model_selection import KFold
from torchvision import transforms
import torch.optim as optim
import itertools
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_fold_1.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "train_dice_epoch", "train_jac_epoch", "val_loss","val_dice", "val_jac"]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs)
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock2D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# data = np.load("./dataISIC2018/ISIC2018_192_256.npz")
# x, y = data["image"], data["mask"]
# fold = kfold.split(x, y)
# n, times = 0, 1
# while n < times:
#     train_ids, val_ids = next(fold)
#     n += 1
# x_train, y_train = x[train_ids], y[train_ids]
# x_test, y_test = x[val_ids], y[val_ids]
# del x, y, data


In [ ]:
data = np.load("./dataISIC2018/ISIC2018_192_256.npz")
x, y = data["image"], data["mask"]
test_size = int((10/100)*x.shape[0])
# test_size = 260
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = test_size, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = test_size, random_state=42)
x_test = np.concatenate([x_test, x_val], axis = 0)
y_test = np.concatenate([y_test, y_val], axis = 0)

# test_size = int((20/100)*x.shape[0])
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=520, random_state=312)
del x, y, data

In [ ]:
# data = np.load("./dataISIC2017/ISIC2017_192x256_train.npz")
# x_train, y_train = data["image"], data["mask"]
# data = np.load("./dataISIC2017/ISIC2017_192x256_test.npz")
# x_test, y_test = data["image"], data["mask"]
# del data

In [ ]:
columns = ['0','1']
df = pd.DataFrame(columns=columns)
_, counts =  np.unique(y_train, return_counts=True)
conts_dict = dict(zip(columns, counts))
df = df.append(conts_dict, ignore_index=True)
label_0 = df['0'].sum()
label_1 = df['1'].sum()
total_labels = label_0 + label_1
n_classes = 2
#Class weights claculation: n_samples / (n_classes * n_samples_for_class)
wt0 = round((total_labels/(n_classes*label_0)), 2)
wt1 = round((total_labels/(n_classes*label_1)), 2)
print("\n Weights are:", wt0, wt1)
class_weight = [wt0, wt1]
# 0.63 2.37


 Weights are: 0.63 2.37


In [ ]:
# train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True,
#                         shuffle=True, num_workers=2,
#                         drop_last=True, prefetch_factor = 8)
# val_dataset = DataLoader(ISICLoader(x_val, y_val, typeData="test"), batch_size=16,
#                           num_workers=2, prefetch_factor=16)
# test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16,
#                           num_workers=2, prefetch_factor=16)
# if torch.cuda.is_available():
#     train_dataset = create_loader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True,
#                         shuffle=True, num_workers=2,
#                         drop_last=True, prefetch_factor = 8)
#     test_dataset = create_loader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16,
#                           num_workers=2, prefetch_factor=16)
# else:
train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True,
                    shuffle=True, num_workers=2,
                    drop_last=True, prefetch_factor = 8)
test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16,
                        num_workers=2, prefetch_factor=16)

In [ ]:
class ActiveFocalLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS, gamma=2):
        """
        class weight should be a list.
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
        self.gamma = gamma
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)
        y_pred = torch.clamp(y_pred, min=1e-6, max=1-1e-6)

        active_focal = - yTrueOnehot * (1-y_pred)**self.gamma * torch.log(y_pred) \
                        - (1 - yTrueOnehot) * y_pred**self.gamma * torch.log(1 - y_pred)
        loss = torch.sum(active_focal, dim=[2, 3]) * self.class_weight
        return torch.sum(loss) / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

class ActiveContourLoss(nn.Module):
    def __init__(self, device, class_weight=[1] * NUM_CLASS):
        """
        class weight should be a list.
        """
        super().__init__()
        self.device = device
        self.class_weight = torch.tensor(class_weight, device=device)
    def forward(self, y_true, y_pred):
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)

        y_pred = torch.clamp(y_pred, min=1e-6, max=1-1e-6)
        active = torch.sum(yTrueOnehot * (1 - y_pred) + (1 - yTrueOnehot) * y_pred, dim=[2, 3])
        loss = torch.sum(active * self.class_weight)
        return loss / (torch.sum(self.class_weight) * y_true.size(0) * y_true.size(2) * y_true.size(3))

In [ ]:
# class DiceLoss(nn.Module):
#     def __init__(self, device):
#         super().__init__()
#         self.device = device
#     def forward(self, y_true, y_pred):
#         yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
#         yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
#         y_pred = y_pred[:,1:]

#         intersection = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
#         cardinality  = torch.sum(yTrueOnehot + y_pred , dim=[1,2,3])
#         loss = 1.0-torch.mean((2. * intersection + 1e-5) / (cardinality + 1e-5))
#         return loss

In [ ]:
# class TverskyLoss(nn.Module):
#     def __init__(self, device, alpha = 0.7):
#         super().__init__()
#         self.device = device
#         self.alpha = alpha
#     def forward(self, y_true, y_pred):
#         yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
#         yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
#         y_pred = y_pred[:,1:]

#         TP = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
#         FN = torch.sum(yTrueOnehot * (1-y_pred), dim=[1,2,3])
#         FP = torch.sum((1-yTrueOnehot) * y_pred, dim=[1,2,3])
#         loss = 1-torch.mean((TP + 1e-5) / (TP + self.alpha*FN + (1-self.alpha)*FP + 1e-5))
#         return loss

In [ ]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = SkipNet()):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score
    def _step(self, batch):
        image, y_true = batch
        y_pred = self.model(image)
        loss_focal = ActiveFocalLoss(self.device, class_weight)(y_true, y_pred)
        loss_contour = ActiveContourLoss(self.device, class_weight)(y_true, y_pred)
        loss = loss_focal + loss_contour
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def configure_optimizers(self):
        optimizer = timm.optim.Nadam(self.parameters(), lr=4e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.7, patience=7, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers


In [ ]:
%cd "/content"
if not os.path.exists("weights"):
    os.mkdir("weights")

/content


In [ ]:
model = SkipNet()
# checkpoint = torch.hub.load_state_dict_from_url(url=model_urls["convnext_tiny_1k"], map_location="cpu", check_hash=True)
# model.load_state_dict(checkpoint["model"], strict=False)

# model.load_state_dict(a["state_dict"], strict=False)
# for layer in list(model.children())[:2]:
#     for parameter in layer.parameters():
#         parameter.requires_grad = False

In [ ]:
segmentor = Segmentor(model=model) #(SegNet(drop_prob=0))
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("./weights/", filename="ckpt{val_dice:0.4f}",
                                                            monitor="val_dice", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar(refresh_rate=1)
logger = HistoryLogger()
swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=200)
PARAMS = {"gpus":1, "benchmark":True, "enable_progress_bar":True, "logger":False,
        #   "callbacks" : [progress_bar],
        #    "overfit_batches" :1,
          "callbacks" : [check_point, swa, logger],
          "log_every_n_steps" :1, "num_sanity_val_steps":0, "max_epochs":250,
        #   "precision":16,
          }

trainer = pl.Trainer(**PARAMS)
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weights/propose.ckpt")
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/weights/ckpt0.9126.ckpt")


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(segmentor, train_dataset, test_dataset)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /content/weights exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | SkipNet | 33.6 M
----------------------------------
33.6 M    Trainable params
0         Non-trainable params
33.6 M    Total params
134.552   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 259: 'val_dice' reached 0.90518 (best 0.90518), saving model to '/content/weights/ckpt0.9052.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 1, global step 518: 'val_dice' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 2, global step 777: 'val_dice' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 3, global step 1036: 'val_dice' was not in top 1
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.save_checkpoint("/content/drive/MyDrive/current.ckpt", weights_only=True)

In [ ]:
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        layer.drop_prob = 0.05

for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        print(layer.drop_prob)

# segmentor.configure_optimizers()

In [ ]:
a = sorted(glob.glob("/content/weights/*"))
from google.colab import files
files.download(a[-1])
# files.download("/content/history_isic_AC.csv")

In [ ]:
trainer.test(segmentor, test_dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_dice           0.9054847359657288
        test_jac            0.8383040428161621
        test_loss           0.09768016636371613
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_dice': 0.9054847359657288,
  'test_jac': 0.8383040428161621,
  'test_loss': 0.09768016636371613}]

In [ ]:
# curn
weight_path = sorted(glob.glob("./weights/*"), reverse=True)
for weight in tqdm(weight_path):
    print(weight)
    segmentor = Segmentor(SegNet(drop_prob=0))
    segmentor = Segmentor.load_from_checkpoint(checkpoint_path=weight)
    dice_score = trainer.test(segmentor, test_dataset)

In [ ]:
weight_path = sorted(glob.glob("./weights/*"), reverse=True)
max_score = 0
best_weight = ""
for weight in tqdm(weight_path):
    segmentor = Segmentor.load_from_checkpoint(checkpoint_path=weight)
    dice_score = trainer.test(segmentor, test_dataset)[0]["test_dice"]
    clear_output()
    print(dice_score)
    if max_score < dice_score:
        max_score = dice_score
        best_weight = weight
print(f"best weight is: {best_weight} with {max_score}")


In [ ]:
def predict(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
weight_path = sorted(glob.glob("./weights/*"), reverse=True)
weight_path

In [ ]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
x_inf = x_test.transpose(0, -1, 1, 2) / 255
segmentor = Segmentor(SegNet(drop_prob=0))
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weights/AC+Global.ckpt")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred = predict(x_inf, segmentor)
torch.cuda.empty_cache()
mask_predict = np.argmax(y_pred, axis=1)
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
print(precision_recall_fscore_support(a, b, average="samples", zero_division=1))
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5)

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5)
print(np.mean(dice1), np.mean(jac))

In [ ]:
# %cd /content/drive/MyDrive/new_paper/ISIC/dataISIC2018
np.savez_compressed("ISIC2018_propose", predicted_mask=mask_predict)

In [ ]:
for i in range(x_test.shape[0]):
    plt.figure(i+1)
    plt.subplot(131), plt.imshow(x_test[i])
    plt.subplot(132), plt.imshow(y_test[i])
    plt.subplot(133), plt.imshow(mask_predict[i])


##Test tine augmentation

In [ ]:
x_test_lr = np.flip(x_test, 2)
x_test_ud = np.flip(x_test, 1)
x_test_lr_ud = np.flip(x_test, [1,2])

In [ ]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
x_pre_origin = x_test.transpose(0, -1, 1, 2) / 255
x_pre_lr = x_test_lr.transpose(0, -1, 1, 2) / 255
x_pre_ud = x_test_ud.transpose(0, -1, 1, 2) / 255
x_pre_lr_ud = x_test_lr_ud.transpose(0, -1, 1, 2) / 255
# segmentor = Segmentor(SkipNet())
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="/content/drive/MyDrive/swa.ckpt")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred_origin = predict(x_pre_origin, segmentor)
y_pred_lr = predict(x_pre_lr, segmentor)
y_pred_ud = predict(x_pre_ud, segmentor)
y_pred_lr_ud = predict(x_pre_lr_ud, segmentor)
torch.cuda.empty_cache()


In [ ]:
y_pred_lr1 = np.flip(y_pred_lr.transpose(0, 2, 3, 1), 2)
y_pred_ud1 = np.flip(y_pred_ud.transpose(0, 2, 3, 1), 1)
y_pred_lr_ud1 = np.flip(y_pred_lr_ud.transpose(0, 2, 3, 1), [1,2])
y_pred_origin1 = y_pred_origin.transpose(0, 2, 3, 1)
y_pred = (y_pred_origin1 + y_pred_lr1 + y_pred_ud1 + y_pred_lr_ud1) / 4

In [ ]:
mask_predict = np.argmax(y_pred, axis=-1)
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
print(precision_recall_fscore_support(a, b, average="samples", zero_division=1))
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5)

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5)
print(np.mean(dice1), np.mean(jac))


(0.9201190416787843, 0.9176587136863313, 0.9074003720437641, None)
0.9074003721964554 0.8418023195033193


In [ ]:
# ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
# [{'test_dice': 0.9054847359657288,
#   'test_jac': 0.8383040428161621,
#   'test_loss': 0.09768016636371613}]

In [ ]:
# for i in range(10,20):
#     plt.figure(i+1)
#     plt.subplot(131), plt.imshow(x_test_lr[i])
#     plt.subplot(132), plt.imshow(x[i])
#     plt.subplot(133), plt.imshow(x_test[i])
